# **Projet UA 3 – Implication de l'IA dans le processus de diagnostic d'une cellule cancereuse**:


## **Introduction**:

### **Informations Complémentaires**



Les caractéristiques sont calculées à partir d'une image numérisée d'un prélèvement par aspiration à l'aiguille fine (PAAF) d'une masse mammaire. Elles décrivent les caractéristiques des noyaux cellulaires présents dans l'image. Quelques images sont disponibles à l'adresse http://www.cs.wisc.edu/~street/images/. Le plan de séparation décrit précédemment a été obtenu par la méthode MSM-T (Multisurface Method-Tree) [KP Bennett, « Decision Tree Construction Via Linear Programming », Actes du 4e congrès de la Midwest Artificial Intelligence and Cognitive Science Society, p. 97-101, 1992], une méthode de classification qui utilise la programmation linéaire pour construire un arbre de décision. Les caractéristiques pertinentes ont été sélectionnées par une recherche exhaustive dans l'espace de 1 à 4 caractéristiques et 1 à 3 plans de séparation. Le programme linéaire utilisé pour obtenir le plan de séparation dans l'espace tridimensionnel est celui décrit dans : [KP Bennett et OL Mangasarian : « Robust Linear Programming Discrimination of Two Linearly Inseparable Sets », Optimization Methods and Software 1, 1992, 23-34]. Cette base de données est également accessible via le serveur FTP du département d'informatique de l'Université de Washington : ftp ftp.cs.wisc.edu cd math-prog/cpo-dataset/machine-learn/WDBC/

--------------------------------------------------------------------------------
En résumé
--------------------------------------------------------------------------------
Au lieu d'utiliser les modèles à base des CNN pour prédire notre cible, des algorithmes ont déjà été utilisé pour traduire les informations sur les images sous forme de données tabulaires. 


### **Description des données**


## Description des variables du dataset *Breast Cancer Wisconsin (Diagnostic)

### 1. Informations générales
- **ID number** : Identifiant unique de l’échantillon.  
- **Diagnosis** : Type de tumeur — **M** (*malignant*, maligne) ou **B** (*benign*, bénigne).



### 2. Caractéristiques cellulaires 

Chaque cellule est décrite à partir de **10 caractéristiques principales** calculées sur l’image numérique du noyau :


**radius** Moyenne des distances du centre au contour (taille globale) 

 **texture** ⁉ Écart-type des niveaux de gris (variation d’intensité)
 
**perimeter** : Longueur du contour cellulaire |

**area** : Surface de la cellule |

**smoothness** : Variation locale du rayon (régularité du contour) |

**compactness** : Degré de compacité *(perimeter² / area - 1)* |

**concavity** : Importance des creux dans le contour |

**concave points** : Nombre de creux (zones concaves) sur le contour |

**symmetry** : Symétrie du noyau |

**fractal dimension** : Complexité du contour (rugosité / forme irrégulière) 



### 3. Trois types de mesures pour chaque caractéristique

Chaque variable ci-dessus est mesurée selon **trois versions** :
- `*_mean` → **Valeur moyenne** sur l’image (ex. `radius_mean`)
- `*_se` → **Erreur standard** ou variation locale (ex. `radius_se`)
- `*_worst` → **Valeur maximale** observée (ex. `radius_worst`)

Ainsi, le dataset contient **10 caractéristiques × 3 mesures = 30 variables**,  
auxquelles s’ajoutent :
- `ID number`
- `Diagnosis`

 **Total : 32 colonnes.**



### Remarque
Les caractéristiques géométriques (*radius, perimeter, area, etc.*) permettent de distinguer les tumeurs bénignes (formes plus régulières et homogènes) des tumeurs malignes (formes irrégulières et plus complexes).




Historique à prendre en compte pour la présentation du projet

1. Comment les données ont été obtenues (imagerie médicale, PAAF, etc.)

    1. Prélèvement du tissu du sein (biopsie)

Lorsqu’une masse suspecte est détectée, un médecin réalise une biopsie :
 un petit échantillon de tissu du sein est prélevé.

Cet échantillon contient des cellules, certaines normales (bénignes) et parfois cancéreuses (malignes).

2. Observation des cellules au microscope

Les cellules prélevées sont ensuite :
-colorées pour bien voir leur forme
- placées sous un microscope numérique
- photographiées avec une caméra haute résolution

Ces images sont la base du dataset.

3. Extraction automatique de caractéristiques (feature extraction)

Un logiciel spécialisé analyse l’image pour mesurer la forme, la structure et la texture des noyaux cellulaires.
Ce sont les fameux attributs du dataset.

Les principales catégories de mesures sont :

*Forme

radius (rayon)

perimeter

area

compactness

concavity

* Texture

texture (variation d’intensité sur l’image)

smoothness (régularité du contour)

* Symétrie et régularité

symmetry

fractal_dimension

concave_points

4. Calcul des 3 versions : mean, se, worst

Pour chaque image, le logiciel analyse plusieurs cellules, pas juste une.
Et il en extrait trois valeurs pour chaque type de mesure :

A mean — moyenne

→ “À quoi ressemblent les cellules en général ?”

B se — erreur standard (standard error) ou variabilité locale (std/sqrt(n) avec n = nombre de cellules)

→ “À quel point les cellules sont-elles différentes les unes des autres ?”

C worst — valeurs les plus extrêmes

→ “Quelles sont les cellules les plus irrégulières ou suspectes ?”

Les tumeurs malignes ont souvent :

des cellules plus grandes

des contours irréguliers

une variabilité élevée

D'où l’importance de ces 3 niveaux.

5. Annotation de la tumeur : B (benign) ou M (malignant)

Finalement, un médecin spécialiste (pathologiste) vérifie la biopsie et attribue la classe cible :

M (Malignant) → tumeur cancéreuse

B (Benign) → tumeur non cancéreuse

Ensuite, cette variable est encodée (ex : B=0, M=1).

 6. Création du dataset final

On rassemble toutes ces informations dans un tableau :

radius_mean	radius_se	radius_worst	…	diagnosis
14.2	0.5	16.3	…	M
8.5	0.2	10.1	…	B

 Chaque ligne = une patiente
Chaque colonne = une caractéristique de l’image




# objectives de la prédiction 


1. Ce que détermine le dataset :
Ce n’est pas pour savoir s’il y a un cancer dans le corps.
 C’est pour déterminer si une tumeur déjà détectée est :
 Bénigne (non cancéreuse)

ou

Maligne (cancéreuse)

Donc :
Oui, c’est de la détection de cancer du sein, mais sur une tumeur déjà observée en imagerie ou palpée par un médecin.

 2. Pourquoi analyser les cellules ?

Quand une tumeur est trouvée, une biopsie est faite pour analyser les cellules au microscope.

Ce dataset utilise l’image du noyau cellulaire pour prédire si la tumeur est :

bénigne → cellules régulières, rondes, stables

maligne → cellules agressives, irrégulières, déformées

Le but du modèle est donc :
“Dire automatiquement si une tumeur du sein est cancéreuse ou non, à partir de l’image de ses cellules.”

3. Si le diagnostic est BENIN (B)

→ Pas de cancer
→ Pas de propagation
→ Souvent on surveille seulement, aucun traitement lourd
→ La tumeur peut rester ou être retirée si elle gêne

4. Si le diagnostic est MALIN (M)
là oui, c'est officiellement un cancer du sein

Une tumeur maligne peut se multiplier, envahir les tissus autour, et se propager dans l’organisme (métastases).

C’est pour ça que la suite est très importante.

5. Prochaine étape quand une tumeur est maligne

Une fois que la biopsie confirme que la tumeur est cancéreuse, le processus médical commence :

Étape 1 – Déterminer le stade du cancer (staging)

Le médecin mesure :

la taille de la tumeur

si elle touche les ganglions

si elle s’est propagée (métastases)

Étape 2 – Bilan complet

Exemples d’examens :

IRM du sein

mammographie

échographie

prise de sang

parfois scanner ou PET-scan

Étape 3 – Choix du traitement

Selon le stade, on peut avoir :

- Chirurgie

Enlever la tumeur (tumorectomie) ou parfois le sein (mastectomie)

- Radiothérapie

Détruire les cellules cancéreuses restantes

- Chimiothérapie

Traitement pour réduire ou éliminer la tumeur

- Hormonothérapie

Si la tumeur réagit aux hormones

- Immunothérapie

Stimuler le système immunitaire pour attaquer les cellules cancéreuses




# une tumeur c'est quoi ? 


1. Une tumeur n’est pas “une seule cellule”

Une tumeur est en réalité :

 un amas (une multiplication) de cellules anormales

qui se divisent trop rapidement et ne respectent plus le fonctionnement normal du tissu.

Donc :

- Une cellule devient anormale

- Elle perd son contrôle de croissance

- Elle commence à se multiplier

- Ces cellules s’accumulent

- L’ensemble forme une tumeur

2. Une cellule anormale → tumeur

Une tumeur commence toujours par une seule cellule qui a muté.
Cette cellule :

ignore les signaux pour arrêter de se diviser

se divise beaucoup plus vite que normal

transmet sa mutation aux cellules filles

Au bout d’un moment, on obtient une masse → la tumeur.

 3. Deux types de tumeurs
 Bénigne

→ cellules un peu anormales mais non agressives
→ elles restent localisées
→ elles ne détruisent pas les tissus autour
→ elles ne se propagent pas dans le corps

 Maligne (cancer)

→ cellules très anormales et agressives
→ elles envahissent les tissus voisins
→ elles peuvent migrer dans le sang ou la lymphe
→ métastases possibles

 4. Ce que le dataset mesure : les cellules qui composent la tumeur

Chaque variable (radius_mean, smoothness_se, compactness_worst, etc.) décrit :

la taille des cellules

leur forme

leur irrégularité

leur densité

leur symétrie

Le but est de voir si les cellules dans l’amas (la tumeur) ressemblent plus à :

des cellules normales / régulières → bénigne

des cellules déformées / agressives → maligne

Donc oui, la tumeur vient d’une cellule, mais ce que l’on analyse dans la réalité (et dans ton dataset) ce sont :

 les caractéristiques des cellules qui composent l’amas tumoral.

Contexte du dataset

Le jeu de données Breast Cancer Wisconsin Diagnostic (ID=17 sur UCI) provient de photographies de cellules épithéliales du sein obtenues par aspiration à l’aiguille fine (FNA).

Chaque ligne du dataset correspond à un échantillon (un prélèvement d’une patiente).

Pour chaque échantillon, on prend une image microscopique.

Sur cette image, on identifie plusieurs cellules et on mesure différentes caractéristiques morphologiques (forme, texture, etc.).

D’où viennent les 30 variables ?

Sur chaque image, on mesure 10 caractéristiques de base des cellules :

radius

texture

perimeter

area

smoothness

compactness

concavity

concave points

symmetry

fractal dimension


pour chaque caractéristique, on calcule trois versions :

Suffixe	Signification	Interprétation
_mean	Moyenne des valeurs mesurées sur toutes les cellules de l’image	donne une idée générale de la forme des cellules
_se	Erreur standard (ou écart-type local)	mesure la variabilité des cellules dans l’échantillon
_worst	Valeur maximale observée parmi les cellules	mesure la cellule la plus anormale du groupe
Donc, pour répondre à ta question :

Est-ce qu’il y a plusieurs photos de cellule, ou bien plusieurs cellules sur une seule photo ?

C’est une seule photo par échantillon,
mais plusieurs cellules sont visibles dessus.

Chaque ligne du dataset représente :

→ une patiente
→ un échantillon de tissu (une photo microscopique)
→ et des statistiques calculées sur plusieurs cellules visibles dans cette photo.

## Les Pré-travaux

la reproductibilité

In [1]:
import pandas as pd
## Pour une affiche plus large de certaines sorties
# Augmenter la largeur d'affichage pour voir toutes les colonnes
pd.set_option('display.max_columns', None)

# Augmenter le nombre de lignes affichées
pd.set_option('display.max_rows', None)

# Désactiver la coupure du texte
pd.set_option('display.max_colwidth', None)

In [2]:
# Pour la reproductibilité (fixer le germe)

import random
import numpy as np
import sklearn

def set_seed_sklearn(seed: int = 42):
    random.seed(seed)                 # Graine Python
    np.random.seed(seed)             # Graine NumPy
    try:
        sklearn.utils.check_random_state(seed) # pour les modèles sklearn utilisant joblib (random forests, etc)
    except:
        pass

# Utilisation
set_seed_sklearn(42)


le chargement des données 

In [3]:
!pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\tareq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

df = pd.concat([X, y], axis=1)

# metadata 
print(breast_cancer_wisconsin_diagnostic.metadata) 
  
# variable information 
df.head()

{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'ID': 230, 'type': 'NATIVE', 'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'venue': 'Electronic imaging', 'year': 1993, 'journal': None, 'DOI': '1

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,radius2,texture2,perimeter2,area2,smoothness2,compactness2,concavity2,concave_points2,symmetry2,fractal_dimension2,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,Diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [5]:
df.shape

(569, 31)